In [4]:
!pip install psycopg2
import psycopg2
from psycopg2 import Error


# Function to print table data in a formatted way
def print_table_data(cursor, table_name):
    # Execute a SELECT statement to fetch all rows from the table
    cursor.execute(f"SELECT * FROM {table_name};")
    # Fetch all the rows as a list of tuples
    data = cursor.fetchall()
    # Get column names from the cursor description
    colnames = [desc[0] for desc in cursor.description]
    # Print the table name
    print(f"\nData in {table_name} table:")
    # Print column names
    print(", ".join(colnames))
    # Print each row of the table
    for row in data:
        print(", ".join(str(x) for x in row))
    # Print an empty line for separation
    print("\n")


# Connect to the PostgreSQL database
try:
    connection = psycopg2.connect(
        dbname="my_database",
        user="postgres",
        password="admin",
        host="localhost",
        port=5433
    )
    print("Connected to the database")

    # Set the isolation level to SERIALIZABLE to ensure ACID properties
    connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)
    cursor = connection.cursor()

    # Execute the create_tables.sql script to set up tables
    with open('C:/Users/akshara/Desktop/cs623/create_tables.sql', 'r') as sql_file:
        cursor.execute(sql_file.read())
    print("Tables created")

    # Print initial table data
    table_names = ["Product", "Depot", "Stock"]
    for table in table_names:
        print_table_data(cursor, table)
    
    # Transaction 2: Delete depot d1 from Depot and Stock
    try:
        cursor.execute("DELETE FROM Stock WHERE deptId = 'd1';")
        cursor.execute("DELETE FROM Depot WHERE deptId = 'd1';")
        connection.commit()
        print("Transaction 2: Successfully deleted depot d1 from Depot and Stock")
        for table in table_names:
            print_table_data(cursor, table)
    except Error as e:
        print(f"Error during Transaction 2: {e}")
        connection.rollback()

    # Transaction 4: Change depot d1 name to dd1 in Depot
    try:
        cursor.execute("UPDATE Depot SET addr = 'dd1' WHERE deptId = 'd1';")
        connection.commit()
        print("Transaction 4: Successfully changed depot d1 name to dd1 in Depot")
        for table in table_names:
            print_table_data(cursor, table)
    except Error as e:
        print(f"Error during Transaction 4: {e}")
        connection.rollback()

    # Transaction 6: Add a depot (d100, Chicago, 100) in Depot and (p1, d100, 100) in Stock
    try:
        cursor.execute("INSERT INTO Depot (deptId, addr, volume) VALUES ('d100', 'Chicago', 100);")
        cursor.execute("INSERT INTO Stock (productId, deptId, quantity) VALUES ('p1', 'd100', 100);")
        connection.commit()
        print("Transaction 6: Successfully added depot (d100, Chicago, 100) in Depot and (p1, d100, 100) in Stock")
        for table in table_names:
            print_table_data(cursor, table)
    except Error as e:
        print(f"Error during Transaction 6: {e}")
        connection.rollback()

    # Other transactions...

finally:
    # Close the cursor and the connection
    if cursor:
        cursor.close()
        print("Cursor closed")
    if connection:
        connection.close()
        print("Connection closed")


Connected to the database
Tables created

Data in Product table:
productid, productname, price
p1, tape, 2.50
p2, tv, 250.00
p3, vcr, 80.00



Data in Depot table:
deptid, addr, volume
d1, New York, 9000
d2, Syracuse, 6000
d4, New York, 2000



Data in Stock table:
productid, deptid, quantity
p1, d1, 1000
p1, d2, -100
p1, d4, 1200
p3, d1, 3000
p3, d4, 2000
p2, d4, 1500
p2, d1, -400
p2, d2, 2000


Transaction 2: Successfully deleted depot d1 from Depot and Stock

Data in Product table:
productid, productname, price
p1, tape, 2.50
p2, tv, 250.00
p3, vcr, 80.00



Data in Depot table:
deptid, addr, volume
d2, Syracuse, 6000
d4, New York, 2000



Data in Stock table:
productid, deptid, quantity
p1, d2, -100
p1, d4, 1200
p3, d4, 2000
p2, d4, 1500
p2, d2, 2000


Transaction 4: Successfully changed depot d1 name to dd1 in Depot

Data in Product table:
productid, productname, price
p1, tape, 2.50
p2, tv, 250.00
p3, vcr, 80.00



Data in Depot table:
deptid, addr, volume
d2, Syracuse, 6000
d4, 